In [22]:
import jax.numpy as jnp
import jax
import numpy as np
import time
import gymnasium as gym
import exciting_environments as excenv
#from exciting_environments import GymWrapper, EnvironmentRegistry
#import jax_dataclasses as jdc
from exciting_environments import EnvironmentRegistry
from dataclasses import fields
from exciting_environments.utils import MinMaxNormalization
jax.config.update("jax_enable_x64", True)
import diffrax
from exciting_environments.pmsm.motor_parameters import MotorVariant

In [14]:
# pmsm_env=EnvironmentRegistry.PMSM.make(saturated=True,motor_variant=excenv.pmsm.MotorVariant.BRUSA,batch_size=2)
# pmsm_env2=EnvironmentRegistry("PMSM-v0").make(saturated=True,motor_variant=excenv.pmsm.MotorVariant("BRUSA"),batch_size=2)
#env=excenv.make('MassSpringDamper-v0',batch_size=2)

In [15]:
pend_env=excenv.PMSM(saturated=False,motor_variant=MotorVariant.BRUSA) #saturated=False,LUT_motor_name="BRUSA"

## Step and Simulate ahead

In [16]:
key=jax.random.PRNGKey(1234)
obs, state = pend_env.reset()

In [17]:
obs,states,last_state=pend_env.sim_ahead(state,jnp.ones((4,2)))
obs

Array([[ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.00484649, -0.0111736 ,  0.        , -0.01061544,  0.98510933,
         0.1719291 ,  0.70525317,  0.70895558],
       [ 0.40377874,  0.04055867,  0.        ,  0.02908998,  0.94088077,
         0.33873792,  0.80709102,  0.58154808],
       [ 0.91747561,  0.07026173,  0.        ,  0.02933019,  0.86863151,
         0.49545867,  0.70140443,  0.56796818],
       [ 1.40103717,  0.08502939,  0.        ,  0.01149924,  0.77051324,
         0.637424  ,  0.6154594 ,  0.60937111]], dtype=float64)

In [18]:
pend_env.generate_rew_trunc_term_ahead(states,jnp.ones((4,2)))

(Array([[0],
        [0],
        [0],
        [0]], dtype=int64),
 Array([[False],
        [False],
        [False],
        [False],
        [ True]], dtype=bool),
 Array([[False],
        [False],
        [False],
        [ True]], dtype=bool))

In [19]:
key=jax.random.PRNGKey(1234)
obs, state = pend_env.reset()
generated_observations = []
generated_actions= []
generated_observations.append(obs)
for i in range(4):
    key,subkey= jax.random.split(key)
    action = jnp.ones(2)#jax.random.uniform(subkey,(2,),minval=-1,maxval=1)
    obs, state = pend_env.step(state, action)
    generated_actions.append(action)
    generated_observations.append(obs)

In [20]:
generated_observations

[Array([0., 0., 0., 0., 1., 0., 0., 0.], dtype=float64, weak_type=True),
 Array([ 0.00484649, -0.0111736 ,  0.        , -0.01061544,  0.98510933,
         0.1719291 ,  0.70525317,  0.70895558], dtype=float64),
 Array([0.40377874, 0.04055867, 0.        , 0.02908998, 0.94088077,
        0.33873792, 0.80709102, 0.58154808], dtype=float64),
 Array([0.91747561, 0.07026173, 0.        , 0.02933019, 0.86863151,
        0.49545867, 0.70140443, 0.56796818], dtype=float64),
 Array([1.40103717, 0.08502939, 0.        , 0.01149924, 0.77051324,
        0.637424  , 0.6154594 , 0.60937111], dtype=float64)]

### Vmapped

In [26]:
batch_size= 2
keys = jax.random.split(jax.random.PRNGKey(0), batch_size)
#envs = [Pendulum(solver=diffrax.Euler(),batch_size=1,static_params={"g": jnp.array(9.81), "l": jnp.array(float(1+i)),  "m": jnp.array(1.0)}) for i in range(4)]
#envs = [excenv.PMSM() for _ in range(batch_size)]
envs = [excenv.PMSM(saturated=False,motor_variant=MotorVariant.BRUSA),excenv.PMSM(saturated=False,motor_variant=MotorVariant.BRUSA)]
#batched_envs = jax.tree.map(lambda *args: jnp.stack(args), *envs)
batched_envs = EnvironmentRegistry("PMSM-v0").make(batch_size=2)
single_env = EnvironmentRegistry("PMSM-v0").make()

In [29]:
keys = jax.random.split(jax.random.PRNGKey(0), batch_size)
# obs, states = jax.vmap(lambda e,k: e.reset(k))(batched_envs,keys)
# print(obs)
obs, states = batched_envs.vmap_reset(keys)
print(obs)

[[ 0.78943966  0.17403345  0.51534122  0.08560387  0.47517159  0.87989315
   0.          0.        ]
 [ 0.80108055  0.6891371  -0.05610045  0.33429225 -0.80029912  0.59960097
   0.          0.        ]]


In [30]:
# state_in=jax.vmap(lambda e,o,k: e.generate_state_from_observation(o,k))(batched_envs,obs,keys)
# print(state_in.physical_state.omega)
state_in=batched_envs.vmap_generate_state_from_observation(obs,keys)
print(state_in.physical_state.i_d)

[-26.32004224 -24.86493112]


In [31]:
actions = jnp.ones((batch_size,5))
obs, states = batched_envs.vmap_reset()
# next_obs, next_states = jax.vmap(
#     lambda e, s, a: e.step(s, a)
# )(batched_envs, states, actions)
# print(next_obs)
next_obs, next_states = batched_envs.vmap_step(states, actions)
print(next_obs)

[[ 0.00405405 -0.0111448   0.         -0.0105901   0.98510933  0.1719291
   0.70525317  0.70895558]
 [ 0.00405405 -0.0111448   0.         -0.0105901   0.98510933  0.1719291
   0.70525317  0.70895558]]


In [32]:
keys = jax.random.split(jax.random.PRNGKey(0), batch_size)
obs, states = batched_envs.vmap_reset()
actions = jnp.ones((batch_size,4,2))
#obs,states,last_state=pend_env.sim_ahead(state,actions=jnp.ones((4,2)))
# next_obs, next_states, last_state = jax.vmap(
#     lambda e, s, a: e.sim_ahead(s, a)
# )(batched_envs, states, actions)
# print(next_obs)
next_obs, next_states, last_state = batched_envs.vmap_sim_ahead(states, actions)
print(next_obs)

[[[ 0.          0.          0.          0.          1.
    0.          0.          0.        ]
  [ 0.00405405 -0.0111448   0.         -0.0105901   0.98510933
    0.1719291   0.70525317  0.70895558]
  [ 0.4022332   0.04063461  0.          0.02916967  0.94088077
    0.33873792  0.80709102  0.58154808]
  [ 0.91554899  0.07041745  0.          0.02945457  0.86863151
    0.49545867  0.70140443  0.56796818]
  [ 1.3992275   0.08528221  0.          0.01159952  0.77051324
    0.637424    0.6154594   0.60937111]]

 [[ 0.          0.          0.          0.          1.
    0.          0.          0.        ]
  [ 0.00405405 -0.0111448   0.         -0.0105901   0.98510933
    0.1719291   0.70525317  0.70895558]
  [ 0.4022332   0.04063461  0.          0.02916967  0.94088077
    0.33873792  0.80709102  0.58154808]
  [ 0.91554899  0.07041745  0.          0.02945457  0.86863151
    0.49545867  0.70140443  0.56796818]
  [ 1.3992275   0.08528221  0.          0.01159952  0.77051324
    0.637424    0.615459

In [33]:
#print(jax.vmap(lambda e, s, a: e.generate_rew_trunc_term_ahead(s, a))(batched_envs, next_states, actions))
batched_envs.vmap_generate_rew_trunc_term_ahead(next_states, actions)

(Array([[[0],
         [0],
         [0],
         [0]],
 
        [[0],
         [0],
         [0],
         [0]]], dtype=int64),
 Array([[[False],
         [False],
         [False],
         [False],
         [ True]],
 
        [[False],
         [False],
         [False],
         [False],
         [ True]]], dtype=bool),
 Array([[[False],
         [False],
         [False],
         [ True]],
 
        [[False],
         [False],
         [False],
         [ True]]], dtype=bool))